In [1]:
import pandas as pd
import docker
import tarfile
from io import BytesIO
from sqlalchemy import create_engine
import os

import warnings
warnings.filterwarnings('ignore')

# Configurações do banco de dados
db_config = {
    'dbname': 'rama',
    'user': 'pcecere',
    'password': '244049',
    'host': 'localhost',
    'port': 5434
}

# Caminho para o arquivo Excel e CSV
excel_file_path = r'X:\Recuperação\Crédito Imobiliário\Acompanhamento\Distribuições ONR – 2024 v1.xlsx'
sheet_name = 'Dados'  # Altere para o nome da aba correta
container_id = '223c59c1268f'  # Substitua pelo ID do seu container

def read_and_process_excel(excel_file_path, sheet_name):
    df = pd.read_excel(excel_file_path, sheet_name=sheet_name)

    rename_df = {
        'Data AF 0.7': 'af_07',
        'Data AF 1.1': 'af_11',
        'Data Distribuição': 'data_distribuicao',
        'Código': 'codigo',
        'Dossiê': 'dossie',
        'Nome do Mutuário': 'adverso',
        'Produto': 'produto',
        'Responsável': 'responsavel',
        'Mês Distr.': 'mes_distribuicao',
        'Temp. Distr.': 'tempo_distribuicao'
    }
    df.rename(columns=rename_df, inplace=True)

    df = df.dropna(subset=['tempo_distribuicao'])

    date_columns = ['af_07', 'af_11', 'data_distribuicao']
    for col in date_columns:
        df[col] = pd.to_datetime(df[col]).dt.date

    df['tempo_distribuicao'] = df['tempo_distribuicao'].astype(int)

    return df

def get_existing_records(db_config):
    engine = create_engine(f"postgresql://{db_config['user']}:{db_config['password']}@{db_config['host']}:{db_config['port']}/{db_config['dbname']}")
    query = "SELECT dossie FROM imobiliario.distribuicao"
    existing_records = pd.read_sql(query, engine)
    return existing_records

def save_df_to_csv(df, csv_file_path):
    df.to_csv(csv_file_path, index=False, encoding='utf-8-sig')

def copy_csv_to_docker_container(csv_file_path, container_id):
    client = docker.from_env()
    container = client.containers.get(container_id)

    with BytesIO() as tar_stream:
        with tarfile.open(fileobj=tar_stream, mode='w') as tar:
            tar.add(csv_file_path, arcname='dados.csv')
        tar_stream.seek(0)
        container.put_archive('/tmp', tar_stream)

def execute_command_in_container(container_id, command):
    client = docker.from_env()
    container = client.containers.get(container_id)
    exec_id = container.exec_run(command)
    print(exec_id.output.decode())

def insert_new_data(df_new_data, db_config, container_id, csv_file_name):
    # Salva o novo dataframe como CSV
    df_new_data.to_csv(csv_file_name, index=False, encoding='utf-8-sig')

    # Copia o CSV para o container Docker
    copy_csv_to_docker_container(csv_file_name, container_id)

    # Carrega os novos dados
    load_csv_command = f"psql -U {db_config['user']} -d {db_config['dbname']} -c \"COPY imobiliario.distribuicao FROM '/tmp/{csv_file_name}' DELIMITER ',' CSV HEADER\""
    execute_command_in_container(container_id, load_csv_command)

if __name__ == "__main__":
    # Ler e processar o Excel
    df = read_and_process_excel(excel_file_path, sheet_name)

    # Obter os registros existentes no banco de dados
    existing_records = get_existing_records(db_config)

    # Comparar os dados novos com os existentes para identificar novos dossiês
    df_new_data = df[~df['dossie'].isin(existing_records['dossie'])]

    if not df_new_data.empty:
        # Se houver novos dados, insira-os no banco
        insert_new_data(df_new_data, db_config, container_id, 'dados.csv')
        os.remove('dados.csv')
        print("Novos dados carregados com sucesso!")
    else:
        print("Nenhum novo dado para carregar.")


COPY 12

Novos dados carregados com sucesso!


In [2]:
import pandas as pd
import docker
import tarfile
from io import BytesIO
from sqlalchemy import create_engine
import os

import warnings
warnings.filterwarnings('ignore')

# Configurações do banco de dados
db_config = {
    'dbname': 'rama',
    'user': 'pcecere',
    'password': '244049',
    'host': 'localhost',
    'port': 5434
}

# Caminho para o arquivo Excel e CSV
excel_file_path = r'X:\Recuperação\Crédito Imobiliário\Acompanhamento\Controle de Intimações Positivas 2024.xlsx'
csv_file_path = r'C:\Users\pedro.cecere\Desktop\dados2.csv'
sheet_name = 'Dados'  # Altere para o nome da aba correta
container_id = '223c59c1268f'  # Substitua pelo ID do seu container

def read_and_process_excel(excel_file_path, sheet_name):
    # Ler a planilha do Excel
    df = pd.read_excel(excel_file_path, sheet_name=sheet_name)

    # Alterar o nome das colunas do df
    rename_df = {
        'Dt. Distr. (AF 1.1)': 'af_11',
        'Dt. Intim. Todos (AF 2.8)': 'af_28',
        'Dt. Ciência Intim.': 'todos_citados',
        'TM Intim. (dias)': 'tm_intimacao',
        'Tipo Intim.': 'tipo_intimacao',
        'Produto': 'produto',
        'Dossiê nº': 'dossie',
        'Adverso': 'adverso',
        'Comarca': 'comarca',
        'UF': 'uf',
        'Últ. Evento WF': 'ultimo_evento_wf',
        'Responsável Intimação': 'resp_intmacao',
        'Responsável Consolidação': 'resp_consolidacao',
        'Mês': 'mes'
    }
    df.rename(columns=rename_df, inplace=True)

    # Alterar tipo de dado das colunas para tipo date
    date_columns = ['af_11', 'af_28', 'todos_citados']
    for col in date_columns:
        df[col] = pd.to_datetime(df[col]).dt.date

    return df

def get_existing_records(db_config):
    # Obter os registros existentes no banco de dados
    engine = create_engine(f"postgresql://{db_config['user']}:{db_config['password']}@{db_config['host']}:{db_config['port']}/{db_config['dbname']}")
    query = "SELECT dossie FROM imobiliario.int_positiva"
    existing_records = pd.read_sql(query, engine)
    return existing_records

def save_df_to_csv(df, csv_file_path):
    # Salvar o DataFrame em um arquivo CSV com encoding UTF-8-SIG
    df.to_csv(csv_file_path, index=False, encoding='utf-8-sig')

def copy_csv_to_docker_container(csv_file_path, container_id):
    # Copiar o arquivo CSV para o container Docker
    client = docker.from_env()
    container = client.containers.get(container_id)

    # Criar um arquivo tar do CSV
    with BytesIO() as tar_stream:
        with tarfile.open(fileobj=tar_stream, mode='w') as tar:
            tar.add(csv_file_path, arcname='dados2.csv')
        tar_stream.seek(0)
        container.put_archive('/tmp', tar_stream)

def execute_command_in_container(container_id, command):
    client = docker.from_env()
    container = client.containers.get(container_id)
    exec_id = container.exec_run(command)
    print(exec_id.output.decode())

def insert_new_data(df_new_data, db_config, container_id, csv_file_name):
    # Define o caminho completo para o arquivo CSV
    csv_file_path = r'C:\Users\pedro.cecere\Desktop' + '\\' + csv_file_name
    
    # Salva o novo dataframe como CSV no caminho especificado
    df_new_data.to_csv(csv_file_path, index=False, encoding='utf-8-sig')

    # Copia o CSV para o container Docker
    copy_csv_to_docker_container(csv_file_path, container_id)

    # Carrega os novos dados
    load_csv_command = f"psql -U {db_config['user']} -d {db_config['dbname']} -c \"COPY imobiliario.int_positiva FROM '/tmp/{csv_file_name}' DELIMITER ',' CSV HEADER\""
    execute_command_in_container(container_id, load_csv_command)

    return csv_file_path
if __name__ == "__main__":
    # Ler e processar o Excel
    df = read_and_process_excel(excel_file_path, sheet_name)

    # Obter os registros existentes no banco de dados
    existing_records = get_existing_records(db_config)

    # Comparar os dados novos com os existentes para identificar novos dossiês
    df_new_data = df[~df['dossie'].isin(existing_records['dossie'])]

    if not df_new_data.empty:
        # Se houver novos dados, insira-os no banco e obtenha o caminho do arquivo CSV
        csv_file_path = insert_new_data(df_new_data, db_config, container_id, 'dados2.csv')
        
        # Remover o arquivo CSV após o uso
        os.remove(csv_file_path)
        print("Novos dados carregados com sucesso!")
    else:
        print("Nenhum novo dado para carregar.")


COPY 3

Novos dados carregados com sucesso!
